<a href="https://colab.research.google.com/github/JeonJongDae/dinner_stock_AI/blob/main/3%EC%A3%BC%EC%B0%A8/%EB%AC%BC%ED%83%80%EA%B8%B0_%EB%B6%88%ED%83%80%EA%B8%B0_%EA%B3%BC%EC%A0%9C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from pandas_datareader import data
import yfinance as yfin
import pandas as pd
from datetime import timedelta
import plotly.graph_objects as go

yfin.pdr_override()

In [4]:
def get_stocks(ticker, start = '2022-03-11'):
    stock_df = data.get_data_yahoo(ticker, start = start)
    #stock_df['Close'].plot()
    return stock_df

In [5]:
stock_df = data.get_data_yahoo('PATH', start = '2022-03-11')
stock_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-11,28.959999,28.959999,26.090,26.209999,26.209999,4941200
2022-03-14,25.920000,26.450001,24.330,24.559999,24.559999,3986900
2022-03-15,24.020000,24.930000,23.930,24.889999,24.889999,5848400
2022-03-16,25.650000,27.940001,25.500,27.820000,27.820000,5142400
2022-03-17,27.219999,28.870001,27.101,28.610001,28.610001,3631200


In [21]:
stock_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-04-06,16.450001,16.754999,16.010000,16.549999,16.549999,4296400
2023-04-10,16.190001,16.570000,16.030001,16.530001,16.530001,6594600
2023-04-11,16.360001,16.799999,16.230000,16.520000,16.520000,4647700
2023-04-12,16.980000,17.020000,16.070000,16.170000,16.170000,3874600
2023-04-13,16.280001,16.650000,16.230000,16.590000,16.590000,3999700


In [23]:
stock_df['Close'][-1]

16.59000015258789

In [6]:
import numpy as np

In [24]:
def get_price_dataframe(num, percent, start_price, bear_bull):
    price_list = [start_price]
    average_price_list = []
    buy_qty =[]
    for i in range(num-1):
        buy_qty.append(i+1)    
        if bear_bull=='0':
           price_list.append(price_list[i] * (1 - percent))
        else:
          price_list.append(price_list[i] * (1 + percent))
    #print(buy_qty)
    df = pd.DataFrame({'매수가': price_list})
    # print("-=====매수가=====")
    # print(df.tail())
    df['매수량'] = np.arange(1,num+1) #buy_qty #1 #range(len(price_list))
    # print("-=====매수량=====")
    # print(df.tail())
    #print(df.head())
    df['보유량'] = df['매수량'].cumsum()
    # print("-=====보유량=====")
    # print(df.tail())
    average_price_list = [df['매수가'][0]]
    #print(len(df))
    for i in range(len(df)-1):        
        average_price = (average_price_list[i] * df['보유량'][i] + df['매수가'][i+1] * df['매수량'][i+1]) / df['보유량'][i+1]
        average_price_list.append(average_price)
    df['매수 후 평균단가'] = average_price_list
    df['원금'] = df['매수 후 평균단가'] * df['보유량'] 
    df['매수 후 평가금액'] = df['매수가'] * df['보유량'] 
    df['수익률'] = df['매수 후 평가금액']/df['원금'] - 1
    return df

In [28]:
def get_sell_dates(plan_df, stock_df,bear_bull):
    first_date = []
    first_cnt =0
    for value in plan_df['매수가']:
        # print(value)
        first_cnt+=1
        index_list = stock_df.query(f'Close >={value} and Close <={value+1}' ).index       
        if len(index_list) == 0 or (len(first_date) > 0 and index_list[-1] - first_date[-1] >= timedelta(days=5)):
            if bear_bull=='1':
               index_list = stock_df.query(f'Close >={value+1}' ).index
            else:
               index_list = stock_df.query(f'Close <={value+1}' ).index
            first_cnt+=1
        #print(index_list)

        if len(first_date) == 0:
            first_date.append(index_list[0])
        else:
            for i in index_list:
                if i > first_date[-1]:
                    # print('i: {}, first_date:{}'.format(i, first_date[-1]))
                    first_date.append(i)
                    break
        # print(first_date[-1])
        # print('-'*10)
        # print(first_date)
    plan_df['매수일자']=first_date
    return first_date

In [9]:

def plot_status(stock_df, plan_df, sell_dates):

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=stock_df.index,
        y=stock_df['Close'],
        name = '종가',
        # connectgaps=True # override default to connect the gaps
    ))
    fig.add_trace(go.Scatter(
        x=stock_df.index,
        y=[plan_df['매수 후 평균단가'].iloc[-1]] * len(stock_df.index),
        name='매수 후 평균단가',
    ))
    fig.add_trace(go.Scatter(
        x=sell_dates,
        y=plan_df['매수가'],#TSLA_df.loc[first_date]['Close'],
        mode="markers+text",
        name="분할매수시점",
        # text=plan_df['매수가'],#["Text D", "Text E", "Text F", '3', '3'],
        textposition="bottom center"
    ))
    fig.show()

In [23]:

def run():#ticker, num, percent, start_price):
    ticker = input('시뮬레이션 할 종목의 티커를 입력하세요.(예:TSLA): ')
    stock_df = get_stocks(ticker)
    #print(stock_df.head())
    num = input('몇 번 분할매수를 진행하시겠습니까? (예:20): ')
    bear_bull = input('물타기를 원하시면 0, 불타기를 원하시면 1을 입력하세요')
    if bear_bull =='1':
       percent = input('몇 % 상승시 분할매수를 진행하시겠습니까? (예:1): ')
    else:
       percent = input('몇 % 하락시 분할매수를 진행하시겠습니까? (예:1): ')
    
    print('{}의 종가 최고가는 {:.2f}입니다.'.format(ticker, stock_df['Close'].max()))
    print('{}의 종가 최저가는 {:.2f}입니다.'.format(ticker, stock_df['Close'].min()))
    print('{}의 오늘 종가는 {:.2f}입니다.'.format(ticker, stock_df['Close'][-1]))
    start_price = input('첫 주가 구매 단가을 얼마로 설정하시겠습니까?(예:214): ')
    
    plan_df = get_price_dataframe(int(num), int(percent)/100, float(start_price),bear_bull)
    #print(plan_df.head())
    sell_dates = get_sell_dates(plan_df, stock_df,bear_bull)
    print(sell_dates)
    print('\n\n')
    print('='*90)
    print('{}의 현재 보유량은 {}, 매수 후 평균단가는 ${:.2f}입니다.'\
      .format(ticker, plan_df.iloc[-1]['보유량'], plan_df.iloc[-1]['매수 후 평균단가']))

    result = (stock_df.iloc[-1]['Close'] * plan_df.iloc[-1]['보유량'] / plan_df.iloc[-1]['원금'] -1) * 100
    print('총 투자 원금은 ${:.2f} 입니다. {} 기준 수익률은 {:.2f}%입니다.'\
        .format(plan_df.iloc[-1]['원금'], stock_df.index[-1], result))    
    
    filename = '{}_planner.xlsx'.format(ticker)
    plan_df.to_excel(filename)
    print('='*90)
    print('※{}이 생성되었습니다.좌측의 생성된 파일을 다운로드 받아주세요'.format(filename))
    plot_status(stock_df, plan_df, sell_dates)

In [30]:
run()

시뮬레이션 할 종목의 티커를 입력하세요.(예:TSLA): PATH
[*********************100%***********************]  1 of 1 completed
몇 번 분할매수를 진행하시겠습니까? (예:20): 5
물타기를 원하시면 0, 불타기를 원하시면 1을 입력하세요1
몇 % 상승시 분할매수를 진행하시겠습니까? (예:1): 5
PATH의 종가 최고가는 31.40입니다.
PATH의 종가 최저가는 10.55입니다.
PATH의 오늘 종가는 16.20입니다.
첫 주가 구매 단가을 얼마로 설정하시겠습니까?(예:214): 12
[Timestamp('2022-09-23 00:00:00'), Timestamp('2022-10-04 00:00:00'), Timestamp('2022-12-02 00:00:00'), Timestamp('2023-01-27 00:00:00'), Timestamp('2023-02-01 00:00:00')]



PATH의 현재 보유량은 15, 매수 후 평균단가는 $13.69입니다.
총 투자 원금은 $205.39 입니다. 2023-04-14 00:00:00 기준 수익률은 18.31%입니다.
※PATH_planner.xlsx이 생성되었습니다.좌측의 생성된 파일을 다운로드 받아주세요
